# 生成每天的测试结果

In [ ]:
model=torch.load("./models/model_saved_emb100_BATCH_1024_LastAggregator_multiclass.pt")
memory,gnn, link_pred,neighbor_loader=model


In [ ]:
ans_5_8=test_day_new(graph_5_8,"graph_5_8")

In [ ]:

ans_5_9=test_day_new(graph_5_9,"graph_5_9")

In [ ]:
ans_5_11=test_day_new(graph_5_11,"graph_5_11")

In [ ]:
ans_5_14=test_day_new(graph_5_14,"graph_5_14")

In [ ]:
ans_5_15=test_day_new(graph_5_15,"graph_5_15")

# 计算IDF字典

这部分的代码使用jinyuan 优化过后的

In [ ]:
# 计算 5-9的IDF值

# share_node_IDF = mp.Manager().dict()

node_set=set()

file_list=[]

file_path="graph_5_9/"
file_l=os.listdir("graph_5_9/")
for i in file_l:
    file_list.append(file_path+i)

# file_path="graph_4_5/"
# file_l=os.listdir("graph_4_5/")
# for i in file_l:
#     file_list.append(file_path+i)

# file_path="graph_4_6/"
# file_l=os.listdir("graph_4_6/")
# for i in file_l:
#     file_list.append(file_path+i)


# file_path="graph_4_7/"
# file_l=os.listdir("graph_4_7/")
# for i in file_l:
#     file_list.append(file_path+i)

for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']):
                node_set.add(str(jdata['srcmsg']))
            if 'netflow' not in str(jdata['dstmsg']):
                node_set.add(str(jdata['dstmsg'])) 


node_list=list(node_set)


# 窗口关系建立函数

In [ ]:
def is_include_key_word(s):
    keywords=[
         'netflow',
        '/dev/pts',
         'proc',
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag


def cal_set_rel(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:

        if is_include_key_word(i) is not True:

            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))

            if (IDF)>4.5 :
                print("node:",i," IDF:",IDF)
                count+=1
    return count

# 异常检测5-14

In [ ]:
# 5-14 直接计算窗口的异常分数

node_IDF=torch.load("node_IDF_5_9")
y_data_5_14=[]
df_list_5_14=[]
# node_set_list=[]
history_list_5_14=[]
tw_que=[]
his_tw={}
current_tw={}
loss_list_5_14=[]


file_path_list=[]
file_path="graph_5_14/"
file_l=os.listdir("graph_5_14/")
for i in file_l:
    file_path_list.append(file_path+i)
    
    
index_count=0
for f_path in (file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_14.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_5_14/")
    # 为当前窗口提取相关信息 用于和历史窗口进行对比
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_14:
        for his_tw in hq:
#             if False: # 默认每个窗口之间没联系
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_list_5_9)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # 判断两个窗口之间是否存在交集
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_14.append(temp_hq)
    index_count+=1
    loss_list_5_14.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

In [ ]:
name_list=[]
for hl in history_list_5_14:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>7:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

# 异常检测5-15

In [ ]:
# 5-15 直接计算窗口的异常分数

# node_IDF=torch.load("node_IDF_5_15")
node_IDF=torch.load("node_IDF_5_9")
y_data_5_15=[]
df_list_5_15=[]
# node_set_list=[]
history_list_5_15=[]
tw_que=[]
his_tw={}
current_tw={}
loss_list_5_15=[]



file_path_list=[]
file_path="graph_5_15/"
file_l=os.listdir("graph_5_15/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in (file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_15.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_5_15/")
    # 为当前窗口提取相关信息 用于和历史窗口进行对比
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_15:
        for his_tw in hq:
#             if False: # 默认每个窗口之间没联系
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'],file_list_5_9)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                # 判断两个窗口之间是否存在交集
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_15.append(temp_hq)
    index_count+=1
    loss_list_5_15.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

In [ ]:
name_list=[]
for hl in history_list_5_15:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>7:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

# label生成

In [ ]:
labels={}
pred_label={}    
    
filelist = os.listdir("graph_5_14")
for f in filelist:
    labels["graph_5_14/"+f]=0
    pred_label["graph_5_14/"+f]=0

filelist = os.listdir("graph_5_15")
for f in filelist:
    labels["graph_5_15/"+f]=0
    pred_label["graph_5_15/"+f]=0

In [ ]:
attack_list=[
    'graph_5_15/2019-05-15 13:58:15.520482252~2019-05-15 14:13:37.257086895.txt',
    'graph_5_15/2019-05-15 14:44:51.773840192~2019-05-15 15:00:26.765466538.txt',
]
#  theia -1  的攻击场景
for i in attack_list:
    labels[i]=1

# 统计attack edge数量

In [ ]:
def keyword_hit(line):
    attack_nodes=[
#             'sshd',
            'sshdlog',
        'shm',
#          'python',
#             'firefox',
        '189.141.204.211',
        '208.203.20.42',
       
#         '',
#         '',
#         '',
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[    
    'graph_5_15/2019-05-15 13:58:15.520482252~2019-05-15 14:13:37.257086895.txt',
    'graph_5_15/2019-05-15 14:44:51.773840192~2019-05-15 15:00:26.765466538.txt',]



In [ ]:
attack_edge_count=0
for fpath in tqdm(files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

# 可视化分析模块

In [ ]:
replace_dic={
    '/run/shm/':'/run/shm/*',
#     '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/':'/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/*',
   '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',    
    '/data/replay_logdb/':'/data/replay_logdb/*', 
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*', 
    '/usr/share/applications/':'/usr/share/applications/*', 
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',     
    '/proc/':'/proc/*', 
     '/stat':'*/stat', 
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*', 
    '/usr/bin/python2.7':'/usr/bin/python2.7/*', 
     '/usr/lib/python2.7':'/usr/lib/python2.7/*', 
'/data/data/org.mozilla.fennec_firefox_dev/cache/':'/data/data/org.mozilla.fennec_firefox_dev/cache/*',
    'UNNAMED':'UNNAMED*',
    '/etc/fonts/':'/etc/fonts/*',
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name

In [ ]:

original_edges_count=0
hash2msg={}
graphs=[]
gg=nx.DiGraph()
count=0
# file_list=os.listdir("./test_day_data4_10_emb100/")
for path in tqdm(attack_list):
#     print(path)
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
#             temp_key=jdata['srcmsg']+jdata['dstmsg']+jdata['edge_type']
#             if temp_key in train_edge_set:
#                 jdata['loss']=(jdata['loss']-train_edge_set[temp_key]) if jdata['loss']>=train_edge_set[temp_key] else 0  
#             jdata['loss']=abs(jdata['loss']-train_edge_set[temp_key])  if temp_key in train_edge_set else jdata['loss']
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1.5*loss_std
#         thr=-99
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
#             if True:  
#                 if "'/home/admin/profile'" in e['srcmsg'] or " '/home/admin/profile'" in e['dstmsg']:
#                     print(e['srcmsg'])
#                     print(e['dstmsg'])
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
                
                hash2msg[str(hashgen(replace_path_name(e['srcmsg'])))]=replace_path_name(e['srcmsg'])
                hash2msg[str(hashgen(replace_path_name(e['dstmsg'])))]=replace_path_name(e['dstmsg'])
                
            #不去除重复节点
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        print(path)
        print("tempg edges:",len(tempg.edges))
        print("tempg nodes:",len(tempg.nodes))
        print("tempg weakly components:",nx.number_weakly_connected_components(tempg))
        
        print("gg edges:",len(gg.edges))
        print("gg nodes:",len(gg.nodes))
        print("gg weakly components:",nx.number_weakly_connected_components(gg))
        print(f"{original_edges_count=}")
#                                 不去除重复节点
#                 gg.add_edge(e['srcnode'],e['dstnode'],loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
#         print(path," line_count:",line_count,"  nodes count:",len(node_set))
                
                
                #         graphs.append(g)

        

In [ ]:
# 这里没有加入太多染色节点，后期手动对比GT文档

def attack_edge_flag(msg):
    attack_edge_type=[        
        '208.203.20.42',
        '189.141.204.211',
        '/var/log/sshdlog',
        '/usr/sbin/sshd',
        '/usr/local/lib/firefox-54.0.1/firefox',        
  
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
    return flag

# 异常检测结果统计分析

沿用其他数据集